In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
import seaborn as sns
from keras import layers
from tensorflow.keras import applications 
from keras.applications import DenseNet121, MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix
import scipy
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

Set random seed for reproducibility.

In [ ]:
np.random.seed(2020)
tf.set_random_seed(2020)

# Loading & Exploration

In [ ]:
train_df = pd.read_csv('../input/valid-and-test-ta/x_train_8.csv')
valid_df = pd.read_csv('../input/valid-and-test-ta/x_valid_8.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)
train_df.head()

In [ ]:
train_df['diagnosis'].hist()
train_df['diagnosis'].value_counts()

In [ ]:
#Mengecek apakah ukuran sudah sesuai
print('Number of train samples: ', train_df.shape[0])
print('Number of test samples: ', valid_df.shape[0])

train_df.head()

In [ ]:
train_df['diagnosis'].hist()
train_df['diagnosis'].value_counts()

### Displaying some Sample Images

In [ ]:
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_df)

# Resize Images

We will resize the images to 224x224, then create a single numpy array to hold the data.

In [ ]:
def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.BILINEAR)
    
    return im

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
N = valid_df.shape[0]
x_val = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(valid_df['id_code'])):
    x_val[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
y_train = train_df['diagnosis']
y_val = valid_df['diagnosis']
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.1,  # set range for random zoom
        rotation_range = 360,
        fill_mode='constant',
        cval=0.,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

# Using original generator
data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE, seed=2019)
# Using Mixup
#mixup_generator = MixupGenerator(x_train, y_train, batch_size=BATCH_SIZE, alpha=0.2, datagen=create_datagen())()

In [ ]:
class_weights = {0: 0.40578462,
                1: 1.98018018,
                2: 0.73348165,
                3: 3.78965517,
                4: 2.47894737}

# Quadratic Weighted Kappa

Quadratic Weighted Kappa (QWK, the greek letter $\kappa$), also known as Cohen's Kappa, is the official evaluation metric. For our kernel, we will use a custom callback to monitor the score, and plot it at the end.

### What is Cohen Kappa?

According to the [wikipedia article](https://en.wikipedia.org/wiki/Cohen%27s_kappa), we have
> The definition of $\kappa$ is:
> $$\kappa \equiv \frac{p_o - p_e}{1 - p_e}$$
> where $p_o$ is the relative observed agreement among raters (identical to accuracy), and $p_e$ is the hypothetical probability of chance agreement, using the observed data to calculate the probabilities of each observer randomly seeing each category.

### How is it computed?

Let's take the example of a binary classification problem. Say we have:

### Creating keras callback for QWK

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, y_val = self.validation_data[:2]
        
        y_pred = self.model.predict(X_val)
        y_pred = np.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model_8.h5')

        return

In [ ]:
kappa_metrics = Metrics()

# Model: MobileNetV2

In [ ]:
densenet = MobileNetV2(
    alpha = 1.3,
    weights='../input/valid-and-test-ta/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.3_224_no_top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
def build_model():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256))
    model.add(layers.Dense(256))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

# Training & Evaluation

In [ ]:
history = model.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[kappa_metrics],
    class_weight=class_weights
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()
history_df.to_csv('CV_8.csv', index = False)

In [ ]:
plt.plot(kappa_metrics.val_kappas)

## Submit

In [ ]:
model.load_weights('model_8.h5')
pred = model.predict(x_val)
y_val_pred = pred
y_val_pred = np.clip(y_val_pred,0,4)
y_val_pred = y_val_pred.astype(int)

labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(valid_df['diagnosis'].astype('int'), y_val_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

In [ ]:
kappa_val = cohen_kappa_score(
            valid_df['diagnosis'].astype('int'),
            y_val_pred, 
            weights='quadratic'
        )
kappa_val

In [ ]:
pred = model.predict(x_train)
y_train_pred = pred
y_train_pred = np.clip(y_train_pred,0,4)
y_train_pred = y_train_pred.astype(int)

labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train_df['diagnosis'].astype('int'), y_train_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

In [ ]:
kappa_train = cohen_kappa_score(
            train_df['diagnosis'].astype('int'),
            y_train_pred, 
            weights='quadratic'
        )
kappa_train